In [15]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from lxml import etree
header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

2018年统计用区划代码和城乡划分代码(截止2018年10月31日)

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/index.html

为了后续拼接中不出现问题，需要将index.html去掉，同样可以访问到刚刚的网站

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/

In [4]:
url='http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/index.html'
header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

# 获取一级目录--l1
省名和一级url

## 请求和响应
因为网页的特殊性，需要对返回的内容进行编码和解码，最终可以得到网页源代码，有两种方法

### 法一：

In [38]:
def get_html(url):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}
    res=requests.get(url,headers=header)
    res.encoding='gbk'   #编码
    h=res.content.decode('gbk') #解码 
    return h


In [40]:
html=get_html(url)
html

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3c.org/TR/1999/REC-html401-19991224/loose.dtd">\r\n<HTML>\r\n<HEAD>\r\n<META content="text/html; charset=gb2312" http-equiv=Content-Type>\r\n<TITLE>2018年统计用区划代码和城乡划分代码</TITLE>\r\n<STYLE type=text/css>\r\n\tBODY {MARGIN: 0px}BODY {FONT-SIZE: 12px}\r\n\tTD {FONT-SIZE: 12px}\r\n\tTH {FONT-SIZE: 12px}\r\n\t.redBig {COLOR: #d00018; FONT-SIZE: 18px; FONT-WEIGHT: bold}\r\n\t.STYLE3 a{COLOR: #fff; text-decoration:none;}\r\n\t.STYLE5 {COLOR: #236fbe; FONT-WEIGHT: bold}\r\n\t.content {LINE-HEIGHT: 1.5; FONT-SIZE: 10.4pt}\r\n\t.tdPading {PADDING-LEFT: 30px}.blue {COLOR: #0000ff}\r\n\t.STYLE6 {COLOR: #ffffff}\r\n\t.a2 {LINE-HEIGHT: 1.5; COLOR: #2a6fbd; FONT-SIZE: 12px}\r\n\ta2:link {LINE-HEIGHT: 1.5; COLOR: #2a6fbd; FONT-SIZE: 12px}\r\n\ta2:hover {LINE-HEIGHT: 1.5; COLOR: #2a6fbd; FONT-SIZE: 12px; TEXT-DECORATION: underline}\r\n\ta2:visited {LINE-HEIGHT: 1.5; COLOR: #2a6fbd; FONT-SIZE: 12px}\r\n</STYLE>\r\n<SCRIPT languag

### 法二：
转码的方式不同而已

In [ ]:
def request_html(url):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}
    res=requests.get(url,headers=header)
    res.encoding=res.apparent_encoding 
    return res.text
html=request_html(url)
html


## 解析
html源代码中，如下样式的a标签的href属性中    存在每个省的url链接：
<a href='11.html'>北京市<br/></a></td><td><a href='12.html'>天津市<br/></a></td><td>
    
1、首先，我们要得到 北京市 11.html， 天津市 12.html 这样的信息

2、其次，进行拼接。拼接的方法中，除了format（），还有+

因为每个省最终的链接并不是11.html，还有一个公共的部分http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/

### re进行解析
set（）方法的作用是将获得的信息转化成一个集合，实现去重的作用，爬虫中常用

In [ ]:
def parse_re(html,addr=''):
    pattern=re.compile("<a href='(.*?)'>(.*?)<")
    ls.extend(list(set(re.findall(pattern,html))))
    for i in ls:
        ls[i]+=addr

### bs进行解析

#### 基本描述
通过该方法可以获得所有的url，['11.html','12.html',……，'http://www.miibeian.gov.cn/']

最后一项不是所要的内容，需要去除

In [65]:
soup=bs(html,'lxml')
tags=soup.find_all('a')
total_data=[]
URL='http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/'
for tag in tags[:-1]:   #剔除掉最后一行的内容
    province=tag.get_text()
    url=tag.get('href')
    data={"province":province,"url":URL+url}
    total_data.append(data)
total_data[:2]

[{'province': '北京市',
  'url': 'http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11.html'},
 {'province': '天津市',
  'url': 'http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12.html'}]

#### 用列表生成式进行简化

In [73]:
soup=bs(html,'lxml')
tags=soup.find_all('a')
addr=''
URL='http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/'
ls=[(URL+i.get('href'),i.get_text()) for i in tags[:-1]]
ls[:2]

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11.html', '北京市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12.html', '天津市')]

用if判断，删除掉后面步骤中不需要的信息
('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省130100000000'),

In [118]:
#定义函数，此时addr=''还没有用，目前可有可无
def parse_bs(html,addr=''):  
    URL='http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/'
    soup=bs(html,'lxml')
    tags=soup.find_all('a')
    l1=[(URL+i.get('href'),addr+i.get_text()) for i in tags[:-1]]
    for i in l1:
        if i[1][-1]=='0':  #如果返回的结果以字符串0结尾，则删除掉，此处没用，后面有用
            l1.remove(i)
    return l1

In [188]:
l1=parse_bs(html)
l1

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11.html', '北京市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12.html', '天津市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13.html', '河北省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/14.html', '山西省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/15.html', '内蒙古自治区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/21.html', '辽宁省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/22.html', '吉林省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/23.html', '黑龙江省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/31.html', '上海市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/32.html', '江苏省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/33.html', '浙江省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/34.html', '安徽省'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/35.html', '福建省'),
 ('http://www.stats.g

# 获取二级目录--l2
 #此时用到之前定义的addr变量
 
<a href="13/1301.html">石家庄市</a>
得到的一级url为 http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12.html'

完整二级url为  http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12/1201.html

## 解析函数中没有if
根据一级url获得二级url过程中，按上面的操作，一个一级url会得到两条数据：第一条数据使我们不需要的，需要在前面定义解析函数的时候，将不需要的数据删除掉

('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省130100000000'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省石家庄市'),

def get_l2(l1):   
    for url_2,addr in l1[:3]:
        html=get_html(url_2)
        l2=parse_bs(html,addr)
    return l2
    

l2=get_l2(l1)
l2[:5]

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省130100000000'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省石家庄市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1302.html',
  '河北省130200000000'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1302.html',
  '河北省唐山市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1303.html',
  '河北省130300000000')]

## 添加if判断后
注意l2.extend（）和l2.append（）的区别

In [198]:
def get_l2(l1):   
    l2=[]  #必须要有，否则只会保存最后一次循环的数据，一级目录获取时，可以不用，
    for url_2,addr in l1[:3]:
        html=get_html(url_2)
        l2.extend(parse_bs(html,addr))
    return l2

In [199]:
l2=get_l2(l1)
l2[:5]

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/1101.html',
  '北京市市辖区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/12/1201.html',
  '天津市市辖区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省石家庄市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1302.html',
  '河北省唐山市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1303.html',
  '河北省秦皇岛市')]

# 获取三级目录--l3
注意：：：三级目录的规律与y一二不同

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/       原始URL 

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11.html   一级

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/1101.html   二级

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110101.html  三级

对应的a标签信息为：<a href="01/110101.html">东城区</a>

## 重新定义l3的解析方法
三级的url共同部分时二级url的前半部分（去掉后9位的部分），不再是之前统一的URL了，

In [132]:
def parse_l3(html,url_3,addr):  
    soup=bs(html,'lxml')
    tags=soup.find_all('a')
    
    l3=[(url_3[:-9]+i.get('href'),addr+i.get_text()) for i in tags[:-1]]
    for i in l3:
        if i[1][-1]=='0':  #如果返回的结果以字符串0结尾，则删除掉，此处没用，后面有用
            l3.remove(i)
    return l3

In [131]:
l2[:3]

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1301.html',
  '河北省石家庄市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1302.html',
  '河北省唐山市'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/1303.html',
  '河北省秦皇岛市')]

## 获取数据

In [200]:
def get_l3(l2):   
    l3=[]
    for url_3,addr in l2[:2]:
        html=get_html(url_3)
        l3.extend(parse_l3(html,url_3,addr))
    return l3

In [210]:
l3=get_l3(l2)
l3

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110101.html',
  '北京市市辖区东城区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110102.html',
  '北京市市辖区西城区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110105.html',
  '北京市市辖区朝阳区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110106.html',
  '北京市市辖区丰台区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110107.html',
  '北京市市辖区石景山区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110108.html',
  '北京市市辖区海淀区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110109.html',
  '北京市市辖区门头沟区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110111.html',
  '北京市市辖区房山区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110112.html',
  '北京市市辖区通州区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/110113.html',
  '北京市市辖区顺义区'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/11011

# 四级目录-l4

<a href="02/130202001.html">学院南路街道办事处</a >

'http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/02/130202.html'  三级

http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/13/02/02/130202001.html 四级

三级去掉后11位的部分+href中的内容

In [162]:
def parse_l4(html,url_4,addr):  
    soup=bs(html,'lxml')
    tags=soup.find_all('a')
    
    l4=[(url_4[:-11]+i.get('href'),addr+i.get_text()) for i in tags[:-1]]
    for i in l4:
        if i[1][-1]=='0':  #如果返回的结果以字符串0结尾，则删除掉，此处没用，后面有用
            l4.remove(i)
    return l4


In [202]:
def get_l4(l3):  
    l4=[]
    for url_4,addr in l3[:2]:
        html=get_html(url_4)
        l4.extend(parse_l4(html,url_4,addr))
    return l4

In [209]:
l4=get_l4(l3)
l4

[('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101001.html',
  '北京市市辖区东城区东华门街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101002.html',
  '北京市市辖区东城区景山街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101003.html',
  '北京市市辖区东城区交道口街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101004.html',
  '北京市市辖区东城区安定门街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101005.html',
  '北京市市辖区东城区北新桥街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101006.html',
  '北京市市辖区东城区东四街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101007.html',
  '北京市市辖区东城区朝阳门街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101008.html',
  '北京市市辖区东城区建国门街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2018/11/01/01/110101009.html',
  '北京市市辖区东城区东直门街道办事处'),
 ('http://www.stats.gov.cn/tjsj/tjbz/tj

# 最后一级目录--l5

最后一级的内容和之前不太一样，下面没有连接了，只需要获取到地址便可，不需要连接，

而且地址不在a标签中，而在tr标签里  
<tr class="villagetr"><td>130202001001</td><td>111</td><td>立新东楼社区居委会</td></tr>

filter(str.isalpha,i.get_text())  

filter的作用是过滤的目的，i.get_text()会得到所有的文本：130202001001立新东楼社区居委会 

前面的数字不是所需要的，需要删除掉，str.isalpha表示匹配的是数字

也可用re模块中的sub函数进行替换

In [165]:
a='130202001001立新东楼社区居委会'
b=filter(str.isalpha,a)
b

In [166]:
list(b)

['立', '新', '东', '楼', '社', '区', '居', '委', '会']

In [172]:
c=''.join(list(filter(str.isalpha,a)))
c

'立新东楼社区居委会'

In [207]:
def parse_l5(html,addr):  
    soup=bs(html,'lxml')
    tags=soup.select('tr.villagetr')
    addr_list=[]
    for i in tags:
        addr_l5=filter(str.isalpha,i.get_text())
        addr_l5=''.join(list(addr_l5))
        addr_list.append(addr+addr_l5)
    return addr_list

def get_l5(l4):   
    l5=[]
    for url_5,addr in l4[:5]:
        html=get_html(url_5)
        l5.extend(parse_l5(html,addr))
    return l5

In [208]:
l5=get_l5(l4)
l5

['北京市市辖区东城区东华门街道办事处多福巷社区居委会',
 '北京市市辖区东城区东华门街道办事处银闸社区居委会',
 '北京市市辖区东城区东华门街道办事处东厂社区居委会',
 '北京市市辖区东城区东华门街道办事处智德社区居委会',
 '北京市市辖区东城区东华门街道办事处南池子社区居委会',
 '北京市市辖区东城区东华门街道办事处黄图岗社区居委会',
 '北京市市辖区东城区东华门街道办事处灯市口社区居委会',
 '北京市市辖区东城区东华门街道办事处正义路社区居委会',
 '北京市市辖区东城区东华门街道办事处甘雨社区居委会',
 '北京市市辖区东城区东华门街道办事处台基厂社区居委会',
 '北京市市辖区东城区东华门街道办事处韶九社区居委会',
 '北京市市辖区东城区东华门街道办事处王府井社区居委会',
 '北京市市辖区东城区景山街道办事处隆福寺社区居委会',
 '北京市市辖区东城区景山街道办事处吉祥社区居委会',
 '北京市市辖区东城区景山街道办事处黄化门社区居委会',
 '北京市市辖区东城区景山街道办事处钟鼓社区居委会',
 '北京市市辖区东城区景山街道办事处魏家社区居委会',
 '北京市市辖区东城区景山街道办事处汪芝麻社区居委会',
 '北京市市辖区东城区景山街道办事处景山东街社区居委会',
 '北京市市辖区东城区景山街道办事处皇城根北街社区居委会',
 '北京市市辖区东城区交道口街道办事处交东社区居委会',
 '北京市市辖区东城区交道口街道办事处福祥社区居委会',
 '北京市市辖区东城区交道口街道办事处大兴社区居委会',
 '北京市市辖区东城区交道口街道办事处府学社区居委会',
 '北京市市辖区东城区交道口街道办事处鼓楼苑社区居委会',
 '北京市市辖区东城区交道口街道办事处菊儿社区居委会',
 '北京市市辖区东城区交道口街道办事处南锣鼓巷社区居委会',
 '北京市市辖区东城区安定门街道办事处交北头条社区居委会',
 '北京市市辖区东城区安定门街道办事处北锣鼓巷社区居委会',
 '北京市市辖区东城区安定门街道办事处国子监社区居委会',
 '北京市市辖区东城区安定门街道办事处钟楼湾社区居委会',
 '北京市市辖区东城区安定门街道办事处宝钞南社区居委会',
 '北京市市辖区东城区安定门街道办事处五道营社区居委会',
 '北京市市辖区东城区安定门街道办事处分司厅社区居